# Week 15 Programming Assignment

The final thing for this semester that we haven't exercised, yet, is working with databases through Python.  In thi final assignment of the semester, you will practice pulling data from either Google Big Query or Snowflake, loading that data into a Pandas data frame, summarizing the data, and then exporting that to an Excel file.

**You will need to use your own Google Big Query or Snowflake account to run this notebook, but you should try to make it configurable so anyone with an account in those technologies could run your notebook with minimal changes.**


Build a notebook with good comments (either in code or in markdown cells).  Then submit your assignment as usual.

### 1. Connect to the data source

Follow the examples provided in the [week15 folder of our GitHub repository](https://github.com/paulboal/hds5210-2023/tree/main/week15) to connect your notebook either to Big Query or Snowflake.

In [1]:
!pip install "snowflake-connector-python[pandas]"

import snowflake.connector
import pandas as pd



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.1.0
    Uninstalling platformdirs-4.1.0:
      Successfully uninstalled platformdirs-4.1.0


In [2]:
# Check if con is defined and close the connection if it is
if 'con' in locals():con.close()

In [3]:
# Create a new connection with snowflake
con = snowflake.connector.connect(
    user='mansipokiya14',
    password='Mansi1408!',
    account='EXYDDML-VY36265',
    database='COVID19_EPIDEMIOLOGICAL_DATA',
    schema='PUBLIC',
    warehouse='COMPUTE_WH',
    session_parameters={
        'QUERY_TAG': 'Demo User Interaction',}
)

In [4]:
# Already have a connection of 'con'
cursor = con.cursor()

In [5]:
# Specify the table name
table_name = 'DEMOGRAPHICS'

In [6]:
# Construct the SELECT query
query = f'SELECT * FROM {table_name}'

In [7]:
# Execute the query
cursor.execute(query)

In [8]:
# Fetching the results into a Pandas DataFrame
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
df = pd.DataFrame(data, columns=columns)

In [9]:
# Print the column names
print("Column Names:")
print(df.columns)

Column Names:
Index(['ISO3166_1', 'ISO3166_2', 'FIPS', 'LATITUDE', 'LONGITUDE', 'STATE',
       'COUNTY', 'TOTAL_POPULATION', 'TOTAL_MALE_POPULATION',
       'TOTAL_FEMALE_POPULATION'],
      dtype='object')


In [10]:
# Print the DataFrame
print("\nDataFrame:")
print(df)


DataFrame:
     ISO3166_1 ISO3166_2   FIPS   LATITUDE   LONGITUDE STATE  \
0           US        CA  06081  37.467348 -122.124828    CA   
1           US        CA  06083  34.388274 -119.490511    CA   
2           US        CA  06085  37.040598 -121.488980    CA   
3           US        CA  06087  36.933939 -121.675387    CA   
4           US        CA  06089  40.796145 -121.415607    CA   
...        ...       ...    ...        ...         ...   ...   
3135        US        NY  36039  42.436161  -73.795047    NY   
3136        US        NY  36041  43.733069  -74.250052    NY   
3137        US        NY  36043  43.131493  -74.774864    NY   
3138        US        NY  36045  44.040206  -75.546141    NY   
3139        US        NY  36047  40.666401  -73.858559    NY   

                    COUNTY  TOTAL_POPULATION  TOTAL_MALE_POPULATION  \
0         San Mateo County            754748                 371545   
1     Santa Barbara County            439395                 220159   
2     

In [11]:
# Close the cursor
cursor.close()

True

### 2. Query some data

Assuming the source database has some data in it or that you can load some data into it from any source, query it.  Then, read that data into a Pandas dataframe.

In [12]:
cursor = con.cursor()

In [13]:
# Specify the table name
table_name = 'DEMOGRAPHICS'

In [14]:
#Make a selection of relevant columns
selected_columns = [
    'STATE', 'COUNTY', 'TOTAL_POPULATION', 'TOTAL_MALE_POPULATION', 'TOTAL_FEMALE_POPULATION'
]

In [15]:
# Construct the SELECT query
columns_query = ', '.join(selected_columns)
query = f'SELECT {columns_query} FROM {table_name}'

In [16]:
# Execute the query
cursor.execute(query)

In [17]:
# Fetching the results into a Pandas DataFrame
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
df = pd.DataFrame(data, columns=columns)

In [18]:
# Print the DataFrame
print(df)

     STATE                      COUNTY  TOTAL_POPULATION  \
0       AK      Aleutians East Borough              3299   
1       AK  Aleutians West Census Area              5714   
2       AK      Anchorage Municipality            299321   
3       AK          Bethel Census Area             17885   
4       AK         Bristol Bay Borough               942   
...    ...                         ...               ...   
3135    PA               Carbon County             64330   
3136    PA               Centre County            159178   
3137    PA              Chester County            512028   
3138    PA              Clarion County             38939   
3139    PA           Clearfield County             81170   

      TOTAL_MALE_POPULATION  TOTAL_FEMALE_POPULATION  
0                      2145                     1154  
1                      3578                     2136  
2                    152681                   146640  
3                      9358                     8527  
4   

In [19]:
# Close the cursor
cursor.close()

True

### 3. Aggregate your data frame

Do some kind of aggregation on your data frame.  Something that makes sense and has some groups to it.  Don't just sum up one column for the entire data frame.  Be more creative than that.

In [20]:
agg_df = df.groupby('STATE').agg({
    'TOTAL_POPULATION': 'sum',
    'TOTAL_MALE_POPULATION': 'sum',
    'TOTAL_FEMALE_POPULATION': 'sum'})

In [21]:
# Print the aggregated DataFrame
print(agg_df)

       TOTAL_POPULATION  TOTAL_MALE_POPULATION  TOTAL_FEMALE_POPULATION
STATE                                                                  
AK               728862                 381037                   347825
AL              4841164                2346193                  2494971
AR              2968472                1456694                  1511778
AZ              6728577                3344106                  3384471
CA             38654206               19200970                 19453236
CO              5359295                2689636                  2669659
CT              3588570                1750270                  1838300
DC               659009                 312629                   346380
DE               934695                 452416                   482279
FL             19934451                9741262                 10193189
GA             10099320                4922471                  5176849
HI              1413673                 709870                  

### 4. Write to Excel

Use Pandas functions to write your summarized data out to a local Excel file.

In [22]:
# Specify the Excel file path
excel_file_path = 'aggregated_data.xlsx'

In [23]:
# Write the DataFrame to Excel
agg_df.to_excel(excel_file_path, index=True)

In [24]:
# Confirm the file creation
print(f'Data has been written to {excel_file_path}')

Data has been written to aggregated_data.xlsx


---

## Submitting Your Work

Submit your work as usual